In [23]:
import pandas as pd
import datetime
import numpy as np

# chart_cv_only_cat

In [2]:
mimic_chart_CV_only_cat = pd.read_csv("../Processed_Data/mimic_chart_CV_only_cat.csv")

In [3]:
mimic_chart_CV_only_cat.head()

,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped,expire_flag,dbsource
0,34360891,30,104557,225176.0,283,2172-10-14 19:00:00,2172-10-14 19:38:00,16018,283_1,NaN,NaN,NaN,NaN,NaN,NotStopd,0,carevue
1,34332845,13,143045,263738.0,283,2167-01-09 00:00:00,2167-01-09 02:29:00,16866,283_1,NaN,NaN,NaN,NaN,NaN,NotStopd,0,carevue
2,34339444,17,161087,257980.0,283,2135-05-10 07:00:00,2135-05-10 06:59:00,15039,283_1,NaN,NaN,NaN,NaN,NaN,NotStopd,0,carevue
3,34337424,17,194023,277042.0,283,2134-12-28 16:00:00,2134-12-28 15:52:00,15652,283_1,NaN,NaN,NaN,NaN,NaN,NotStopd,0,carevue
4,34337425,17,194023,277042.0,284,2134-12-28 16:00:00,2134-12-28 15:52:00,15652,284_1,NaN,NaN,NaN,NaN,NaN,NotStopd,0,carevue


In [4]:
chart_cv_cat_itemids_value_map_list = pd.read_csv("./data/chart_cv_cat_itemids_unique_value_map.csv")

In [6]:
chart_cv_cat_itemids_value_map_list.head()

,itemid,true_unique_value,mapped_value
0,184,4 Spontaneously,184_1
1,184,1 No Response,184_2
2,184,3 To speech,184_3
3,184,2 To pain,184_4
4,723,5 Oriented,723_1


* From block In[8] and In[9] we get that the transformed tabular representation for the chart_cv categorical itemids' part has 447 columns combined with all the one-hot encoded values of each categorical itemid, the name of columns is the variable "expected_cols_chart_cv_cat_itemids_tabular_rep"

In [8]:
chart_cv_cat_itemids_value_map_list.shape

(447, 3)

In [9]:
expected_cols_chart_cv_cat_itemids_tabular_rep = list(chart_cv_cat_itemids_value_map_list["mapped_value"].unique())
len(expected_cols_chart_cv_cat_itemids_tabular_rep)

447

In [12]:
mimic_icu_adm = pd.read_csv("./data/mimic_icuadmittime.csv")

# FUNC: Cur_Patient_24hr_cat_itemids_value_chart_cv(hadm_id, recording_range): 
* the result returned by this function is the final version of chart_cv categorical itemids part


In [36]:
# "recording_range" == 24 (hrs) currently
def Cur_Patient_24hr_cat_itemids_value_chart_cv(hadm_id, recording_range):
    cur_dataset_af_gb_hadmid = mimic_chart_CV_only_cat.groupby("hadm_id")
    cur_adm_recordings = cur_dataset_af_gb_hadmid.get_group(hadm_id)[["hadm_id", "itemid", "charttime", "value"]]
    cur_icu_adm_time = mimic_icu_adm[mimic_icu_adm["hadm_id"] == hadm_id]["intime"].unique()[0]
    icu_adm_timestamp = datetime.datetime.timestamp(datetime.datetime.strptime(cur_icu_adm_time,"%Y-%m-%d %H:%M:%S"))
    cur_adm_recordings[["charttimestamp"]] = cur_adm_recordings[["charttime"]].apply((lambda x: datetime.datetime.timestamp(datetime.datetime.strptime(x["charttime"],"%Y-%m-%d %H:%M:%S"))), axis=1)
    cur_adm_recordings[["charttime_interval_after_icu_adm"]] = (cur_adm_recordings[["charttimestamp"]] - icu_adm_timestamp) / 3600
    cur_adm_recordings[["charttime_interval_after_icu_adm"]] = cur_adm_recordings[["charttime_interval_after_icu_adm"]].astype(int) + 1 # "+1" because astype(int) will only keep the integer part of the float number
    # for example: charttime_interval_after_icu_adm == 0.2, actually this belongs to the first one-hour window, so the result should be 1, not 0 (if returned directly by astype(int))
    cur_adm_recordings.drop(columns=["charttime", "charttimestamp"], inplace=True) # columns: "hadm_id", "itemid", "charttime_interval_after_icu_adm", "value" are left 
    cur_adm_recordings = cur_adm_recordings[cur_adm_recordings["charttime_interval_after_icu_adm"] <= 24] # only keep the recordings in the first 24-hour window range
    cur_adm_recordings.sort_values(by="charttime_interval_after_icu_adm", inplace=True, ignore_index=True)
    cur_adm_itemids = cur_adm_recordings["itemid"].unique()
    cur_adm_gb_itemid = cur_adm_recordings.groupby("itemid")
    final_cur_adm_recordings = 0
    for j, itemid in enumerate(cur_adm_itemids):
        cur_item = cur_adm_gb_itemid.get_group(itemid)
        cur_item_first_value_one_window = cur_item.groupby("charttime_interval_after_icu_adm").first()
        if j == 0:
            final_cur_adm_recordings = cur_item_first_value_one_window
        else:
            final_cur_adm_recordings = pd.concat([final_cur_adm_recordings, cur_item_first_value_one_window])
    final_cur_adm_recordings.reset_index(inplace=True)
    final_cur_adm_recordings = final_cur_adm_recordings.pivot(index="charttime_interval_after_icu_adm", columns="itemid", values="value")
    for cat_itemid in final_cur_adm_recordings.columns:
        cur_itemid_dummies = pd.get_dummies(final_cur_adm_recordings[cat_itemid])
        final_cur_adm_recordings = pd.concat([final_cur_adm_recordings, cur_itemid_dummies], axis=1)
        final_cur_adm_recordings.drop(columns=[cat_itemid], inplace=True, axis=1)
    left_one_hot_cat_itemid_chart_cv = set(expected_cols_chart_cv_cat_itemids_tabular_rep)-set(final_cur_adm_recordings.columns)
    if len(left_one_hot_cat_itemid_chart_cv) > 0:
        added_itemids_recordings = np.zeros((final_cur_adm_recordings.shape[0], len(left_one_hot_cat_itemid_chart_cv))).astype(int)
        final_cur_adm_recording_added = pd.DataFrame(data=added_itemids_recordings, columns=list(left_one_hot_cat_itemid_chart_cv), index=final_cur_adm_recordings.index)
        final_cur_adm_recordings = pd.concat([final_cur_adm_recordings, final_cur_adm_recording_added], axis=1)
    final_cur_adm_recordings = final_cur_adm_recordings[expected_cols_chart_cv_cat_itemids_tabular_rep]
    
    # imputation for chart_cv_cat_itemids: only insert those missing 1-hr time windows' rows with all the values: 0
    if final_cur_adm_recordings.shape[0] != recording_range:
        cur_index = set(final_cur_adm_recordings.index)
        expected_index = set(range(1, recording_range+1))
        missed_index = expected_index - cur_index
        interpolate_rows = pd.DataFrame(np.zeros((len(missed_index), final_cur_adm_recordings.shape[1])), 
                                        columns=final_cur_adm_recordings.columns, index=missed_index)
        final_cur_adm_recordings = pd.concat([final_cur_adm_recordings, interpolate_rows]).sort_index(ascending=True)
    return final_cur_adm_recordings



In [37]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(Cur_Patient_24hr_cat_itemids_value_chart_cv(104557, 24))

,184_1,184_2,184_3,184_4,723_1,723_2,723_3,723_4,723_5,723_6,617_1,617_2,617_3,617_4,617_5,617_6,617_7,617_8,454_1,454_2,454_3,454_4,454_5,454_6,674_1,674_2,674_3,674_4,674_5,674_6,478_1,478_2,478_3,478_4,478_5,478_6,478_7,478_8,478_9,80_1,80_2,80_3,80_4,80_5,704_1,704_2,704_3,704_4,704_5,704_6,704_7,704_8,644_1,644_2,644_3,644_4,210_1,210_2,210_3,210_4,210_5,210_6,210_7,210_8,31_1,31_2,31_3,31_4,31_5,31_6,31_7,159_1,159_2,159_3,159_4,159_5,32_1,32_2,32_3,32_4,32_5,84_1,84_2,84_3,84_4,82_1,82_2,82_3,82_4,85_1,85_2,85_3,85_4,86_1,86_2,86_3,86_4,83_1,83_2,83_3,88_1,88_2,88_3,88_4,8480_1,8480_2,8480_3,8480_4,8480_5,8480_6,156_1,156_2,156_3,156_4,156_5,156_6,156_7,156_8,1046_1,1046_2,1046_3,8381_1,8381_2,8381_3,8381_4,8381_5,8381_6,8381_7,8381_8,547_1,547_2,547_3,547_4,637_1,637_2,637_3,637_4,637_5,637_6,637_7,637_8,128_1,128_2,128_3,128_4,128_5,128_6,550_1,550_2,550_3,550_4,550_5,550_6,550_7,550_8,8400_1,8400_2,8400_3,8400_4,8400_5,8400_6,549_1,549_2,549_3,549_4,549_5,549_6,549_7,549_8,8456_1,8456_2,8456_3,8456_4,8456_5,8456_6,8456_7,8456_8,1125_1,1125_2,1125_3,1125_4,1125_5,1125_6,1337_1,1337_2,1337_3,1337_4,1337_5,1337_6,1337_7,1087_1,1087_2,1087_3,1087_4,1087_5,187_1,187_2,187_3,187_4,187_5,187_6,187_7,1622_1,1622_2,1622_3,1623_1,1623_2,8401_1,8401_2,8401_3,8401_4,8401_5,8401_6,1427_1,1427_2,1427_3,1427_4,1427_5,49_1,49_2,49_3,49_4,72_1,72_2,72_3,477_1,477_2,477_3,642_1,642_2,642_3,925_1,925_2,925_3,925_4,925_5,925_6,925_7,925_8,925_9,924_1,924_2,1484_1,1484_2,8367_1,8367_2,8367_3,167_1,167_2,167_3,167_4,167_5,169_1,169_2,168_1,168_2,168_3,168_4,168_5,170_1,170_2,170_3,170_4,170_5,230_1,230_2,230_3,230_4,230_5,283_1,283_2,283_3,283_4,283_5,283_6,283_7,127_1,127_2,127_3,127_4,127_5,548_1,548_2,548_3,605_1,605_2,605_3,605_4,605_5,605_6,605_7,605_8,604_1,604_2,604_3,604_4,604_5,604_6,604_7,604_8,623_1,623_2,623_3,623_4,623_5,623_6,623_7,722_1,722_2,722_3,722_4,40_1,40_2,40_3,40_4,40_5,40_6,40_7,40_8,585_1,585_2,585_3,585_4,585_5,585_6,585_7,585_8,166_1,166_2,166_3,166_4,166_5,610_1,610_2,610_3,8467_1,8467_2,8467_3,8467_4,8467_5,8467_6,8467_7,8467_8,584_1,584_2,584_3,584_4,1089_1,1089_2,1089_3,1089_4,1089_5,8466_1,8466_2,8466_3,8466_4,1047_1,1047_2,1047_3,8382_1,8382_2,8382_3,8382_4,8382_5,8382_6,732_1,732_2,732_3,732_4,657_1,657_2,657_3,657_4,8477_1,8477_2,8477_3,8477_4,8477_5,8477_6,200_1,200_2,200_3,200_4,200_5,200_6,200_7,200_8,200_9,8388_1,8388_2,8388_3,8388_4,8388_5,203_1,203_2,203_3,203_4,203_5,203_6,660_1,660_2,660_3,660_4,8476_1,8476_2,8476_3,8476_4,8476_5,8476_6,284_1,284_2,284_3,284_4,284_5,284_6,284_7,1043_1,1043_2,1043_3,1043_4,1043_5,1043_6,1043_7,236_1,236_2,236_3,236_4,236_5,392_1,392_2,392_3,392_4,392_5,392_6,392_7,392_8,392_9,183_1,183_2,183_3,8428_1,8428_2,8428_3,197_1,197_2,197_3,197_4
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [38]:
Cur_Patient_24hr_cat_itemids_value_chart_cv(104557, 24).shape

(24, 447)

# chart_mv_only_cat

In [32]:
mimic_chart_MV_only_cat = pd.read_csv("../Processed_Data/mimic_chart_MV_only_cat.csv")

In [33]:
mimic_chart_MV_only_cat.head()

,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped,expire_flag,dbsource
0,255752583,7936,144371,247302,227688,2157-11-12 17:02:00,2157-11-12 17:08:00,14757.0,1,1.0,NaN,0.0,0.0,NaN,NaN,0,metavision
1,255752587,7936,144371,247302,227688,2157-11-12 18:12:00,2157-11-12 18:15:00,14757.0,1,1.0,NaN,0.0,0.0,NaN,NaN,0,metavision
2,255753601,8081,175774,226082,227688,2125-07-11 21:21:00,2125-07-11 21:21:00,16436.0,1,1.0,NaN,0.0,0.0,NaN,NaN,1,metavision
3,255753615,8081,175774,226082,227688,2125-07-11 21:55:00,2125-07-11 22:17:00,16436.0,1,1.0,NaN,0.0,0.0,NaN,NaN,1,metavision
4,255753631,8081,175774,226082,227688,2125-07-11 23:54:00,2125-07-11 23:55:00,16436.0,1,1.0,NaN,0.0,0.0,NaN,NaN,1,metavision


In [35]:
# each patient in chart_mv should have the same cat_itemid cols, which is variable: "expected_cols_chart_mv_cat_itemids_tabular_rep"
expected_cols_chart_mv_cat_itemids_tabular_rep = list(mimic_chart_MV_only_cat["itemid"].unique())
print(len(expected_cols_chart_mv_cat_itemids_tabular_rep))

37


# Because categorical itemids in chart_mv are all binary itemids, so no need to one-hot encoded them.
# The basic mind behind tranformation for chart_mv_cat_itemids is same as transformation of lab itemids

In [45]:
# "recording_range" == 24 (hrs) currently
def Cur_Patient_24hr_cat_itemids_value_chart_mv(hadm_id, recording_range):
    cur_dataset_af_gb_hadmid = mimic_chart_MV_only_cat.groupby("hadm_id")
    cur_adm_recordings = cur_dataset_af_gb_hadmid.get_group(hadm_id)[["hadm_id", "itemid", "charttime", "value"]]
    cur_icu_adm_time = mimic_icu_adm[mimic_icu_adm["hadm_id"] == hadm_id]["intime"].unique()[0]
    icu_adm_timestamp = datetime.datetime.timestamp(datetime.datetime.strptime(cur_icu_adm_time,"%Y-%m-%d %H:%M:%S"))
    cur_adm_recordings[["charttimestamp"]] = cur_adm_recordings[["charttime"]].apply((lambda x: datetime.datetime.timestamp(datetime.datetime.strptime(x["charttime"],"%Y-%m-%d %H:%M:%S"))), axis=1)
    cur_adm_recordings[["charttime_interval_after_icu_adm"]] = (cur_adm_recordings[["charttimestamp"]] - icu_adm_timestamp) / 3600
    cur_adm_recordings[["charttime_interval_after_icu_adm"]] = cur_adm_recordings[["charttime_interval_after_icu_adm"]].astype(int) + 1 # "+1" because astype(int) will only keep the integer part of the float number
    # for example: charttime_interval_after_icu_adm == 0.2, actually this belongs to the first one-hour window, so the result should be 1, not 0 (if returned directly by astype(int))
    cur_adm_recordings.drop(columns=["charttime", "charttimestamp"], inplace=True) # columns: "hadm_id", "itemid", "charttime_interval_after_icu_adm", "value" are left 
    cur_adm_recordings = cur_adm_recordings[cur_adm_recordings["charttime_interval_after_icu_adm"] <= 24] # only keep the recordings in the first 24-hour window range
    cur_adm_recordings.sort_values(by="charttime_interval_after_icu_adm", inplace=True, ignore_index=True)
    cur_adm_itemids = cur_adm_recordings["itemid"].unique()
    cur_adm_gb_itemid = cur_adm_recordings.groupby("itemid")
    final_cur_adm_recordings = 0
    for j, itemid in enumerate(cur_adm_itemids):
        cur_item = cur_adm_gb_itemid.get_group(itemid)
        cur_item_first_value_one_window = cur_item.groupby("charttime_interval_after_icu_adm").first()
        if j == 0:
            final_cur_adm_recordings = cur_item_first_value_one_window
        else:
            final_cur_adm_recordings = pd.concat([final_cur_adm_recordings, cur_item_first_value_one_window])
    final_cur_adm_recordings.reset_index(inplace=True)
    final_cur_adm_recordings = final_cur_adm_recordings.pivot(index="charttime_interval_after_icu_adm", columns="itemid", values="value")
    
    left_cat_itemid_chart_mv = set(expected_cols_chart_mv_cat_itemids_tabular_rep)-set(final_cur_adm_recordings.columns)
    if len(left_cat_itemid_chart_mv) > 0:
        final_cur_adm_recording_added = pd.DataFrame(np.nan, columns=list(left_cat_itemid_chart_mv), index=final_cur_adm_recordings.index)
        final_cur_adm_recordings = pd.concat([final_cur_adm_recordings, final_cur_adm_recording_added], axis=1)
    final_cur_adm_recordings = final_cur_adm_recordings[expected_cols_chart_mv_cat_itemids_tabular_rep]
    
    # imputation for chart_cv_cat_itemids: only insert those missing 1-hr time windows' rows with all the values: NaN
    if final_cur_adm_recordings.shape[0] != recording_range:
        cur_index = set(final_cur_adm_recordings.index)
        expected_index = set(range(1, recording_range+1))
        missed_index = expected_index - cur_index
        interpolate_rows = pd.DataFrame(np.nan, columns=final_cur_adm_recordings.columns, index=missed_index)
        final_cur_adm_recordings = pd.concat([final_cur_adm_recordings, interpolate_rows]).sort_index(ascending=True)
    return final_cur_adm_recordings



In [46]:
test_tranformation_chart_mv_cat = Cur_Patient_24hr_cat_itemids_value_chart_mv(175774, 24)
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(test_tranformation_chart_mv_cat)

,227688,227367,227292,227368,227349,225103,225106,225110,225113,225092,225094,225118,225122,225124,225126,225074,225076,225087,225184,225185,225187,225210,225067,225070,225078,225313,224641,224168,228099,228100,228236,226137,226107,226180,226138,226179,226169
1,1.0,1.0,NaN,NaN,NaN,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN
2,1.0,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,1.0,NaN,NaN,NaN,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
test_tranformation_chart_mv_cat = Cur_Patient_24hr_cat_itemids_value_chart_mv(144371, 24)
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(test_tranformation_chart_mv_cat)

,227688,227367,227292,227368,227349,225103,225106,225110,225113,225092,225094,225118,225122,225124,225126,225074,225076,225087,225184,225185,225187,225210,225067,225070,225078,225313,224641,224168,228099,228100,228236,226137,226107,226180,226138,226179,226169
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
3,1.0,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,NaN,1.0,0.0,NaN,1.0,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.0,NaN
4,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
10,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
